In [76]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [77]:
# Collect Data
df = pd.read_csv('tata.csv')

# Data Preprocessing
scaler = MinMaxScaler()
data = scaler.fit_transform(df[['Close']])
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

def create_dataset(data, look_back):
    X, y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back), 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

look_back = 30
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

In [78]:
# Building the Model
model = Sequential([
  LSTM(50, input_shape=(look_back, 1)),
  Dropout(0.2),
  Dense(1)
])

In [79]:
# Training the Model
model.compile(optimizer='adam', loss= 'mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
6/6 [==============================] - 2s 17ms/step - loss: 0.1803
Epoch 2/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0516
Epoch 3/100
6/6 [==============================] - 0s 17ms/step - loss: 0.0164
Epoch 4/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0228
Epoch 5/100
6/6 [==============================] - 0s 21ms/step - loss: 0.0135
Epoch 6/100
6/6 [==============================] - 0s 29ms/step - loss: 0.0146
Epoch 7/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0133
Epoch 8/100
6/6 [==============================] - 0s 30ms/step - loss: 0.0136
Epoch 9/100
6/6 [==============================] - 0s 31ms/step - loss: 0.0117
Epoch 10/100
6/6 [==============================] - 0s 28ms/step - loss: 0.0113
Epoch 11/100
6/6 [==============================] - 0s 27ms/step - loss: 0.0112
Epoch 12/100
6/6 [==============================] - 0s 25ms/step - loss: 0.0115
Epoch 13/100
6/6 [==============================]

In [80]:
# Evaluating the Model
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])
rmse_train = np.sqrt(np.mean((train_predict - y_train)**2))
rmse_test = np.sqrt(np.mean((test_predict - y_test)**2))

1/1 [==============================] - 0s 25ms/step


In [81]:
# Making Predictions
last_60_days = data[-60:]
X_test = []
X_test.append(last_60_days)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print("Predicted Price: ", pred_price)

1/1 [==============================] - 0s 438ms/step
Predicted Price:  [[103.9654]]
